<a href="https://colab.research.google.com/github/tkdoon/mask_nomask_app/blob/main/yolov7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

yolov7

In [ ]:
%cd /content

In [ ]:
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7

In [ ]:
%cd /content/yolov7

requirements.txtをyamlだけに変えたほうがいいらしい

In [ ]:
!pip install -r requirements.txt

In [ ]:
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-e6.pt -P ./checkpoints

モデルを試すだけ

In [ ]:
!python detect.py --source inference/images/horses.jpg \
  --weights checkpoints/yolov7-e6.pt \
  --conf 0.25 --img-size 1280 --device 0
from IPython.display import Image
Image("runs/detect/exp/horses.jpg")

マスクした人のデータセット

In [ ]:
!mkdir dataset
%cd dataset

In [ ]:
!curl -L "https://public.roboflow.com/ds/m07FII2ClC?key=W8TVU9SlqE" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

In [ ]:
%cd  /content/yolov7

dataファルダにmask_wearing.yamlを作り，以下をコピペ[リンクテキスト](https://)

\# train and val data directory\
train: ./dataset/train\
val: ./dataset/valid

\# number of classes\
nc: 2

\# class names\
names: ['1つ目のクラス名', '2つ目のクラス名']


In [ ]:
!python train.py --workers 2 --batch-size 8 \
  --data data/mask_wearing.yaml \
  --cfg cfg/training/yolov7.yaml \
  --weights 'checkpoints/yolov7.pt' \
  --name yolov7-mask \
  --hyp data/hyp.scratch.p6.yaml \
  --epochs 100 \
  --device 0

↓food recognition

In [ ]:
%cd /content

In [ ]:
!git clone https://github.com/tkdoon/mask_nomask_app.git

In [ ]:
%cd /content/yolov7/dataset/
!mkdir train
!mkdir valid
%cd train
!mkdir labels
!mkdir images
%cd ../valid
!mkdir labels
!mkdir images

In [ ]:
%cd /content/mask_nomask_app/ObjectDetection/

img_directory_path = '/content/drive/MyDrive/Photos/'\
train_img_path = '/content/yolov7/dataset/train/images/'\
valid_img_path = '/content/yolov7/dataset/valid/images/'\
train_txt_path = '/content/yolov7/dataset/train/labels/'\
valid_txt_path = '/content/yolov7/dataset/valid/labels/'\
をsplit_img_make_txt.pyに設定

In [ ]:
!python split_img_make_txt.py

ここからやればOK

In [ ]:
%cd /content/yolov7/

\# train and val data directory\
train: ./dataset/train\
val: ./dataset/valid\
\# number of classes\
nc: 16\
\# class names\
names: ['bangbangji-tofu','chiken_shiodare','hamburg_oroshi_sauce','hanjuku_tamago','ichigo_loveberry','karamiso_butadon','karamiso_ramen',
        'kebab_rice','mugi_gohan','pho','potato_daizu_meat_curry','rice','sakedon','satsuma_chiken_liver','shiromi_fry_tarutaru','waguri_monburan']

をfood_recog.yamlにコピペ

In [ ]:
%cp -f -r /content/drive/MyDrive/dataset /content/yolov7/ 

In [ ]:
!python train.py --workers 2 --batch-size 8 \
  --data data/food_recog.yaml \
  --cfg cfg/training/yolov7.yaml \
  --weights 'checkpoints/yolov7.pt' \
  --name yolov7-food \
  --hyp data/hyp.scratch.p6.yaml \
  --epochs 100 \
  --device 0

In [ ]:
%pip install nvidia-pyindex
%pip install onnx-graphsurgeon
%pip install onnx onnxruntime onnxsim onnx-tf

In [ ]:
%cd /content/yolov7/runs/train/yolov7-food5/weights/

In [ ]:
!python /content/yolov7/export.py --weights last.pt --grid --end2end --simplify \
        --topk-all 100 --iou-thres 0.65 --conf-thres 0.35 --img-size 640 640 --max-wh 640